In [1]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Contents and why we need this lab\n",
    "\n",
    "This lab is about implementing neural networks yourself in NumPy before we start using other frameworks which hide some of the computation from you. It builds on the first lab where you derived the equations for neural network forward and backward propagation and gradient descent parameter updates."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# External sources of information\n",
    "\n",
    "1. Jupyter notebook. You can find more information about Jupyter notebooks [here](https://jupyter.org/). It will come as part of the [Anaconda](https://www.anaconda.com/) Python installation. \n",
    "2. [NumPy](https://numpy.org/). Part of Anaconda distribution. If you already know how to program most things about Python and NumPy can be found through Google search. \n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# This notebook will follow the next steps:\n",
    "\n",
    "1. Data generation\n",
    "2. Initialization of parameters\n",
    "3. Definition of activation functions   \n",
    "4. A short explanation of numpy's einsum function\n",
    "5. Forward pass\n",
    "6. Backward pass (backward pass and finite differences)\n",
    "7. Training loop \n",
    "8. Testing your model\n",
    "9. Further extensions"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Create an artificial dataset to play with\n",
    "\n",
    "We create a non-linear 1d regression task. The generator has support various noise levels and it creates train, validation and test sets. You can modify it yourself if you want more or less challenging tasks."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {},
   "outputs": [],
   "source": [
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "\n",
    "np.random.seed(42)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 3,
   "metadata": {},
   "outputs": [],
   "source": [
    "def data_generator(noise=0.1, n_samples=300, D1=True):\n",
    "    # Create covariates and response variable\n",
    "    if D1:\n",
    "        X = np.linspace(-3, 3, num=n_samples).reshape(-1,1) # 1-D\n",
    "        np.random.shuffle(X)\n",
    "        y = np.random.normal((0.5*np.sin(X[:,0]*3) + X[:,0]), noise) # 1-D with trend\n",
    "    else:\n",
    "        X = np.random.multivariate_normal(np.zeros(3), noise*np.eye(3), size = n_samples) # 3-D\n",
    "        np.random.shuffle(X)    \n",
    "        y = np.sin(X[:,0]) - 5*(X[:,1]**2) + 0.5*X[:,2] # 3-D\n",
    "\n",
    "    # Stack them together vertically to split data set\n",
    "    data_set = np.vstack((X.T,y)).T\n",
    "    \n",
    "    train, validation, test = np.split(data_set, [int(0.35*n_samples), int(0.7*n_samples)], axis=0)\n",
    "    \n",
    "    # Standardization of the data, remember we do the standardization with the training set mean and standard deviation\n",
    "    train_mu = np.mean(train, axis=0)\n",
    "    train_sigma = np.std(train, axis=0)\n",
    "    \n",
    "    train = (train-train_mu)/train_sigma\n",
    "    validation = (validation-train_mu)/train_sigma\n",
    "    test = (test-train_mu)/train_sigma\n",
    "    \n",
    "    x_train, x_validation, x_test = train[:,:-1], validation[:,:-1], test[:,:-1]\n",
    "    y_train, y_validation, y_test = train[:,-1], validation[:,-1], test[:,-1]\n",
    "\n",
    "    return x_train, y_train,  x_validation, y_validation, x_test, y_test"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 4,
   "metadata": {},
   "outputs": [],
   "source": [
    "D1 = True\n",
    "x_train, y_train,  x_validation, y_validation, x_test, y_test = data_generator(noise=0.5, D1=D1)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 5,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "image/png": "...\n",
      "text/plain": [
       "<Figure size 432x288 with 1 Axes>"
      ]
     },
     "metadata": {
      "needs_background": "light"
     },
     "output_type": "display_data"
    }
   ],
   "source": [
    "if D1:\n",
    "    plt.scatter(x_train[:,0], y_train);\n",
    "    plt.scatter(x_validation[:,0], y_validation);\n",
    "    plt.scatter(x_test[:,0], y_test);\n",
    "else:\n",
    "    plt.scatter(x_train[:,1], y_train);\n",
    "    plt.scatter(x_validation[:,1], y_validation);\n",
    "    plt.scatter(x_test[:,1], y_test);"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Initialization"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "The steps to create a feed forward neural network are the following:\n",
    "\n",
    "1. **Number of hidden layer and hidden units**. We have to define the number of hidden units in each layer. We are going to save these numbers in a list \"L\" that is going to start with our input dimensionality (the number of features in X) and is going to finish with our output dimensionality (the size of Y). Anything in between these values are going to be hidden layers and the number of hidden units in each hidden layer is defined by the researcher. Remember that for each unit in each layer (besides the first one, according to our list L) there is a bias term.\n",
    "2. **Activation functions** for each hidden layer. Each hidden layer in your list must have an activation function (it can also be the linear activation which is equivalent to identity function). The power of neural networks comes from non-linear activation functions that learn representations (features) from the data allowing us to learn from it. \n",
    "3. **Parameter initialization**. We will initialize the weights to have random values. This is done in practice by drawing pseudo random numbers from a Gaussian or uniform distribution. It turns out that for deeper models we have to be careful about how we scale the random numbers. This will be the topic of the exercise below. For now we will just use unit variance Gaussians.  \n",
    "\n",
    "Our initialization will work as follows: \n",
    "\n",
    "For each layer of the neural network defined in L, initialize a matrix of weights of size (units_in, units_out) from a random normal distribution [np.random.normal()](https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.random.normal.html) and save them in a list called \"layers\". For each layer in our neural network, initialize a matrix of weights of size (1, units_out) as above and save them in a list called \"bias\". The function should return a tuple (layers, bias). The length of our lists must be len(L)-1."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 8,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Initialize neural network:\n",
    "# the NN is a tuple with a list with weights and list with biases\n",
    "def init_NN(L):\n",
    "    \"\"\"\n",
    "    Function that initializes our feed-forward neural network. \n",
    "    Input: \n",
    "    L: list of integers. The first element must be equal to the number of features of x and the last element \n",
    "        must be the number of outputs in the network.\n",
    "    Output:\n",
    "    A tuple of:\n",
    "    weights: a list with randomly initialized weights of shape (in units, out units) each. The units are the ones we defined in L.\n",
    "        For example, if L = [2, 3, 4] layers must be a list with a first element of shape (2, 3) and a second elemtn of shape (3, 4). \n",
    "        The length of layers must be len(L)-1\n",
    "    biases: a list with randomly initialized biases of shape (1, out_units) each. For the example above, bias would be a list of length\n",
    "        2 with a first element of shape (1, 3) and a second element of shape (1, 4).\n",
    "    \"\"\"\n",
    "    weights = [] # empty list of weights\n",
    "    biases  = [] # an empty list of biases\n",
    "    for i in range(len(L)-1): \n",
    "        weights.append(np.random.normal(loc=0.0, scale=1.0, size=[L[i],L[i+1]])) # mean =0 ,std = 1 ,\n",
    "        biases.append(np.random.normal(loc=0.0, scale=1.0, size=[1, L[i+1]]))     \n",
    "        \n",
    "    return (weights, biases) \n",
    "\n",
    "# Initialize the unit test neural network:\n",
    "# Same steps as above but we will not initialize the weights randomly.\n",
    "def init_NN_UT(L):\n",
    "    weights = []\n",
    "    biases  = []\n",
    "    for i in range(len(L)-1):\n",
    "        weights.append(np.ones((L[i],L[i+1]))) \n",
    "        biases.append(np.ones((1, L[i+1])))     \n",
    "        \n",
    "    return (weights, biases)\n",
    "\n",
    "# Initializer the unit test neural network\n",
    "L_UT  = [3, 5, 1]\n",
    "NN_UT = init_NN_UT(L_UT)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Exercise a) Print all network parameters\n",
    "\n",
    "Make a function that prints all parameters (weights and biases) with information about in which layer the parameters are."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 9,
   "metadata": {},
   "outputs": [],
   "source": [
    "def print_para(weights,biases):\n",
    "    # printing the weights\n",
    "    i=1\n",
    "    for array in weights:\n",
    "        print(\"------------------------------------------------\")\n",
    "        print(\"weight layer number\",i)\n",
    "        print(\"------------------------------------------------\")\n",
    "        # now i m inside the array i have to loop over the elements\n",
    "        y=1\n",
    "        for row in array :\n",
    "            k=1\n",
    "            for column in row:\n",
    "                print(\"    weight position:\")\n",
    "                print(\"          from unit#\",y,\"in unit layer#\",i,)\n",
    "                print(\"          to unit#\",k,\"in unit layer#\",i+1,)\n",
    "                print(\"    value=\")\n",
    "                print(\"         \",column)\n",
    "                k=k+1\n",
    "            y=y+1\n",
    "        i=i+1\n",
    "    #printing the biases\n",
    "    i=1\n",
    "    for array in biases:#bias is a list of arrays\n",
    "        print(\"------------------------------------------------\")\n",
    "        print(\"biases for layer\",i)\n",
    "        print(\"------------------------------------------------\")\n",
    "        y=1\n",
    "        for row in array:\n",
    "            k=1\n",
    "            for element in row:\n",
    "                print(\"to unit\",k,\"in layer\",i+1)\n",
    "                print(\"  value=\",element)\n",
    "                k=k+1\n",
    "        \n",
    "            y=y+1\n",
    "        i=i+1"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 23,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "------------------------------------------------\n",
      "weight layer number 1\n",
      "------------------------------------------------\n",
      "    weight position:\n",
      "          from unit# 1 in unit layer# 1\n",
      "          to unit# 1 in unit layer# 2\n",
      "    value=\n",
      "          0.02769093161621279\n",
      "    weight position:\n",
      "          from unit# 1 in unit layer# 1\n",
      "          to unit# 2 in unit layer# 2\n",
      "    value=\n",
      "          0.1273654785371559\n",
      "    weight position:\n",
      "          from unit# 1 in unit layer# 1\n",
      "          to unit# 3 in unit layer# 2\n",
      "    value=\n",
      "          -0.3614228394269773\n",
      "    weight position:\n",
      "          from unit# 1 in unit layer# 1\n",
      "          to unit# 4 in unit layer# 2\n",
      "    value=\n",
      "          -0.23849336738405827\n",
      "    weight position:\n",
      "          from unit# 1 in unit layer# 1\n",
      "          to unit# 5 in unit layer# 2\n",
      "    value=\n",
      "          -0.4581099028313764\n",
      "    weight position:\n",
      "          from unit# 2 in unit layer# 1\n",
      "          to unit# 1 in unit layer# 2\n",
      "    value=\n",
      "          -0.6259929302544003\n",
      "    weight position:\n",
      "          from unit# 2 in unit layer# 1\n",
      "          to unit# 2 in unit layer# 2\n",
      "    value=\n",
      "          -0.3819102223687616\n",
      "    weight position:\n",
      "          from unit# 2 in unit layer# 1\n",
      "          to unit# 3 in unit layer# 2\n",
      "    value=\n",
      "          -0.4134512388176997\n",
      "    weight position:\n",
      "          from unit# 2 in unit layer# 1\n",
      "          to unit# 4 in unit layer# 2\n",
      "    value=\n",
      "          -0.8285837710187354\n",
      "    weight position:\n",
      "          from unit# 2 in unit layer# 1\n",
      "          to unit# 5 in unit layer# 2\n",
      "    value=\n",
      "          -0.5451691993318439\n",
      "    weight position:\n",
      "          from unit# 3 in unit layer# 1\n",
      "          to unit# 1 in unit layer# 2\n",
      "    value=\n",
      "          -2.5027595216622154\n",
      "    weight position:\n",
      "          from unit# 3 in unit layer# 1\n",
      "          to unit# 2 in unit layer# 2\n",
      "    value=\n",
      "          0.6846422015551685\n",
      "    weight position:\n",
      "          from unit# 3 in unit layer# 1\n",
      "          to unit# 3 in unit layer# 2\n",
      "    value=\n",
      "          0.39025326898314217\n",
      "    weight position:\n",
      "          from unit# 3 in unit layer# 1\n",
      "          to unit# 4 in unit layer# 2\n",
      "    value=\n",
      "          -0.08490883249583231\n",
      "    weight position:\n",
      "          from unit# 3 in unit layer# 1\n",
      "          to unit# 5 in unit layer# 2\n",
      "    value=\n",
      "          0.23732438475974324\n",
      "------------------------------------------------\n",
      "weight layer number 2\n",
      "------------------------------------------------\n",
      "    weight position:\n",
      "          from unit# 1 in unit layer# 2\n",
      "          to unit# 1 in unit layer# 3\n",
      "    value=\n",
      "          2.0433204896823054\n",
      "    weight position:\n",
      "          from unit# 2 in unit layer# 2\n",
      "          to unit# 1 in unit layer# 3\n",
      "    value=\n",
      "          1.619612155437311\n",
      "    weight position:\n",
      "          from unit# 3 in unit layer# 2\n",
      "          to unit# 1 in unit layer# 3\n",
      "    value=\n",
      "          0.9330121403883498\n",
      "    weight position:\n",
      "          from unit# 4 in unit layer# 2\n",
      "          to unit# 1 in unit layer# 3\n",
      "    value=\n",
      "          -0.7617349157278194\n",
      "    weight position:\n",
      "          from unit# 5 in unit layer# 2\n",
      "          to unit# 1 in unit layer# 3\n",
      "    value=\n",
      "          -0.564557003269575\n",
      "------------------------------------------------\n",
      "biases for layer 1\n",
      "------------------------------------------------\n",
      "to unit 1 in layer 2\n",
      "  value= 0.2735774031386282\n",
      "to unit 2 in layer 2\n",
      "  value= -0.7240017966042989\n",
      "to unit 3 in layer 2\n",
      "  value= -0.5325803022204045\n",
      "to unit 4 in layer 2\n",
      "  value= 0.5283166096303764\n",
      "to unit 5 in layer 2\n",
      "  value= 0.20750348515913597\n",
      "------------------------------------------------\n",
      "biases for layer 2\n",
      "------------------------------------------------\n",
      "to unit 1 in layer 3\n",
      "  value= -1.0020467236287343\n"
     ]
    }
   ],
   "source": [
    "L=[3,5,1]\n",
    "(weights, biases)=init_NN(L)\n",
    "print_para(weights,biases)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Advanced initialization schemes\n",
    "\n",
    "If we are not careful with initialization we can run into trouble with in both the forward and backward passes. We have random weights with random +/- sign so the signal we pass forward will also be random and zero on average. However, the absolute size of the signal may grow or shrink from layer to layer depending upon the absolute scale of random weights. A statistical analysis of this effect and the same effect for the backward pass are presented in these two papers: [Glorot initialization](http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf) and [He initialization](https://arxiv.org/pdf/1502.01852v1.pdf). \n",
    "\n",
    "The result of the analyses are proposals for how to make the initialization such that the variance of the signals (forward and backward) are kept constant when propagating layer to layer. The exact expressions depend upon the activation function used.\n",
    "\n",
    "We define $n_{in}$ and $n_{out}$ as the number of input units and output units of a particular layer. \n",
    "\n",
    "In the linked paper, Glorot and Bengio propose that for tanh activation functions the following two alternative initializations:\n",
    "\n",
    "$$w_{ij} \\sim U \\bigg[ -\\sqrt{\\frac{6}{(n_{in} + n_{out})}}, \\, \\sqrt{\\frac{6}{(n_{in} + n_{out})}} \\bigg]$$\n",
    "\n",
    "$$w_{ij} \\sim N \\bigg( 0, \\, \\frac{2}{(n_{in} + n_{out})} \\bigg) \\ . $$\n",
    "\n",
    "Here $U[a,b]$ is a uniform distribution in the interval $a$ to $b$ and $N(\\mu,\\sigma^2)$ is a Gaussian distribution with mean $\\mu$ and variance $\\sigma^2$.\n",
    "\n",
    "He et.al. proposes for Rectified Linear Unit activations (ReLU) the following initialization:\n",
    "\n",
    "$$w_{ij} \\sim U \\bigg[ -\\sqrt{\\frac{6}{n_{in}}}, \\, \\sqrt{\\frac{6}{n_{in}}} \\bigg]$$\n",
    "\n",
    "$$w_{ij} \\sim N \\bigg( 0, \\, \\frac{2}{n_{in}} \\bigg) \\ . $$"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Exercise b) Glorot and He initialization\n",
    "\n",
    "Implement these initialization schemes by modifying the code given below.\n",
    "\n",
    "**NOTE:** The Gaussian is defined as $N( \\mu, \\, \\sigma^{2})$ but Numpy takes $\\sigma$ as argument.\n",
    "\n",
    "Explain briefly how you would test numerically that these initializations have the sought after property. Hint: See plots in Glorot paper."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 54,
   "metadata": {},
   "outputs": [],
   "source": [
    "import math\n",
    "## Glorot\n",
    "def init_NN_glorot_Tanh(L, uniform=False):\n",
    "    \"\"\"\n",
    "    Initializer using the glorot initialization scheme\n",
    "    \"\"\"\n",
    "    weights = []\n",
    "    biases  = []\n",
    "    for i in range(len(L)-1):\n",
    "        if(i==0):\n",
    "            nin=0\n",
    "            nout=L[i]*L[i+1]\n",
    "        elif(i==len(L)-1):\n",
    "            nin=L[i-1]*L[i]\n",
    "            nout= 0\n",
    "        else:\n",
    "            nin=L[i-1]*L[i]\n",
    "            nout=L[i]*L[i+1]\n",
    "       \n",
    "        if uniform:\n",
    "            bound = math.sqrt(6/(nin+nout)) # <- replace with proper initialization\n",
    "            weights.append(np.random.uniform(low=-bound, high=bound, size=[L[i],L[i+1]])) \n",
    "            biases.append(np.random.uniform(low=-bound, high=bound, size=[1, L[i+1]]))  \n",
    "        else:\n",
    "            std = math.sqrt(2/(nin+nout)) #<- replace with proper initialization\n",
    "            weights.append(np.random.normal(loc=0.0, scale=std, size=[L[i],L[i+1]])) \n",
    "            biases.append(np.random.normal(loc=0.0, scale=std, size=[1, L[i+1]]))       \n",
    "        \n",
    "    return (weights, biases)\n",
    "\n",
    "## He\n",
    "def init_NN_he_ReLU(L, uniform=False):\n",
    "    \"\"\"\n",
    "    Initializer using the He initialization scheme\n",
    "    \"\"\"\n",
    "    weights = []\n",
    "    biases  = []\n",
    "    for i in range(len(L)-1):\n",
    "        nin=L[i]*L[i+1]\n",
    "        nout=L[i+1]*L[i+2]\n",
    "        if uniform:\n",
    "            bound = math.sqrt(6/nin) # <- replace with proper initialization\n",
    "            weights.append(np.random.uniform(low=-bound, high=bound, size=[L[i],L[i+1]])) \n",
    "            biases.append(np.random.uniform(low=-bound, high=bound, size=[1, L[i+1]]))  \n",
    "        else:\n",
    "            std = math.sqrt(2/nin) # <- replace with proper initialization\n",
    "            weights.append(np.random.normal(loc=0.0, scale=std, size=[L[i],L[i+1]])) \n",
    "            biases.append(np.random.normal(loc=0.0, scale=std, size=[1, L[i+1]]))       \n",
    "        \n",
    "    return (weights, biases)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Activation functions\n",
    "\n",
    "To have a full definition of the neural network, we must define an activation function for every layer in our list L (again, exluding the first term, which is the number of input dimensions). Several activation functions have been proposed and have different characteristics. Here, we will implement the linear activation function (the identity function), the sigmoid activation function (squeeshes the outcome of each neuron into the $[0, 1]$ range), the Hyperbolic Tangent (Tanh) that squeeshes the outcome of each neuron to $[-1, 1]$ and the Rectified Linear Unit (ReLU). \n",
    "\n",
    "We will also include the derivative in the function. We need this in order to do our back-propagation algorithm. Don't rush, we will get there soon. For any neural network, save the activation functions in a list. This list must be of size len(L)-1."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Linear activation"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 18,
   "metadata": {},
   "outputs": [],
   "source": [
    "def Linear(x, derivative=False):\n",
    "    \"\"\"\n",
    "    Computes the element-wise Linear activation function for an array x\n",
    "    inputs:\n",
    "    x: The array where the function is applied\n",
    "    derivative: if set to True will return the derivative instead of the forward pass\n",
    "    \"\"\"\n",
    "    \n",
    "    if derivative:              # Return the derivative of the function evaluated at x\n",
    "        return np.ones_like(x)\n",
    "    else:                       # Return the forward pass of the function at x\n",
    "        return x"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Sigmoid activation"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 19,
   "metadata": {},
   "outputs": [],
   "source": [
    "def Sigmoid(x, derivative=False):\n",
    "    \"\"\"\n",
    "    Computes the element-wise Sigmoid activation function for an array x\n",
    "    inputs:\n",
    "    x: The array where the function is applied\n",
    "    derivative: if set to True will return the derivative instead of the forward pass\n",
    "    \"\"\"\n",
    "    f = 1/(1+np.exp(-x))\n",
    "    \n",
    "    if derivative:              # Return the derivative of the function evaluated at x\n",
    "        return f*(1-f)\n",
    "    else:                       # Return the forward pass of the function at x\n",
    "        return f"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Hyperbolic Tangent activation"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 20,
   "metadata": {},
   "outputs": [],
   "source": [
    "def Tanh(x, derivative=False):\n",
    "    \"\"\"\n",
    "    Computes the element-wise Sigmoid activation function for an array x\n",
    "    inputs:\n",
    "    x: The array where the function is applied\n",
    "    derivative: if set to True will return the derivative instead of the forward pass\n",
    "    \"\"\"\n",
    "    f = (np.exp(x)-np.exp(-x))/(np.exp(x)+np.exp(-x))\n",
    "    \n",
    "    if derivative:              # Return the derivative of the function evaluated at x\n",
    "        return 1-f**2\n",
    "    else:                       # Return the forward pass of the function at x\n",
    "        return f"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Rectifier linear unit (ReLU)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 21,
   "metadata": {},
   "outputs": [],
   "source": [
    "def ReLU(x, derivative=False):\n",
    "    \"\"\"\n",
    "    Computes the element-wise Rectifier Linear Unit activation function for an array x\n",
    "    inputs:\n",
    "    x: The array where the function is applied\n",
    "    derivative: if set to True will return the derivative instead of the forward pass\n",
    "    \"\"\"\n",
    "    \n",
    "    if derivative:              # Return the derivative of the function evaluated at x\n",
    "        return (x>0).astype(int)\n",
    "    else:                       # Return the forward pass of the function at x\n",
    "        return np.maximum(x, 0)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Visualization"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Now that we have defined our activation functions we can visualize them to see what they look like:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 22,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "image/png": "...\n",
      "text/plain": [
       "<Figure size 360x360 with 1 Axes>"
      ]
     },
     "metadata": {
      "needs_background": "light"
     },
     "output_type": "display_data"
    }
   ],
   "source": [
    "x = np.linspace(-6, 6, 100)\n",
    "units = {\n",
    "    \"Linear\": lambda x: Linear(x),\n",
    "    \"Sigmoid\": lambda x: Sigmoid(x),\n",
    "    \"ReLU\": lambda x: ReLU(x),\n",
    "    \"tanh\": lambda x: Tanh(x)\n",
    "}\n",
    "\n",
    "plt.figure(figsize=(5, 5))\n",
    "[plt.plot(x, unit(x), label=unit_name, lw=2) for unit_name, unit in units.items()]\n",
    "plt.legend(loc=2, fontsize=16)\n",
    "plt.title('Our activation functions', fontsize=20)\n",
    "plt.ylim([-2, 5])\n",
    "plt.xlim([-6, 6])\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Exercise c) Glorot initialization for all activation functions\n",
    "\n",
    "Implement a function by adding to the code snippet below that can take network L and list of activations function as argument and return a Glorot initialized network.  Hint: [This blog post](https://mmuratarat.github.io/2019-02-25/xavier-glorot-he-weight-init) gives a table for the activation functions we use here.\n",
    "\n",
    "Briefly explain in words how these how these values are calculated."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  },
  {
   "cell_type": "code",
   "execution_count": 58,
   "metadata": {},
   "outputs": [],
   "source": [
    "import math\n",
    "## Glorot\n",
    "def init_NN_Glorot(L, activations, uniform=False):\n",
    "    \"\"\"\n",
    "    Initializer using the glorot initialization scheme\n",
    "    \"\"\"\n",
    "    weights = []\n",
    "    biases  = []\n",
    "    for i in range(len(L)-1):\n",
    "        if(i==0):\n",
    "            nin=0\n",
    "            nout=L[i]*L[i+1]\n",
    "        elif(i==len(L)-1):\n",
    "            nin=L[i-1]*L[i]\n",
    "            nout= 0\n",
    "        else:\n",
    "            nin=L[i-1]*L[i]\n",
    "            nout=L[i]*L[i+1]\n",
    "        # for the sigmoid function\n",
    "        if(activations[i]==Sigmoid):\n",
    "            if uniform:\n",
    "                bound = 4*math.sqrt(6/(nin+nout)) \n",
    "                weights.append(np.random.uniform(low=-bound, high=bound, size=[L[i],L[i+1]])) \n",
    "                biases.append(np.random.uniform(low=-bound, high=bound, size=[1, L[i+1]]))  \n",
    "            else:\n",
    "                std = 4*math.sqrt(2/(nin+nout)) \n",
    "                weights.append(np.random.normal(loc=0.0, scale=std, size=[L[i],L[i+1]])) \n",
    "                biases.append(np.random.normal(loc=0.0, scale=std, size=[1, L[i+1]]))\n",
    "        #for the RELU       \n",
    "        elif(activations[i]==ReLU):\n",
    "            if uniform:\n",
    "                bound = math.sqrt(2)*math.sqrt(6/(nin+nout)) \n",
    "                weights.append(np.random.uniform(low=-bound, high=bound, size=[L[i],L[i+1]])) \n",
    "                biases.append(np.random.uniform(low=-bound, high=bound, size=[1, L[i+1]]))  \n",
    "            else:\n",
    "                std = math.sqrt(2)*math.sqrt(2/(nin+nout)) \n",
    "                weights.append(np.random.normal(loc=0.0, scale=std, size=[L[i],L[i+1]])) \n",
    "                biases.append(np.random.normal(loc=0.0, scale=std, size=[1, L[i+1]]))\n",
    "        #for the Tanh\n",
    "        elif(activations[i]==Tanh):\n",
    "            if uniform:\n",
    "                bound = math.sqrt(6/(nin+nout)) \n",
    "                weights.append(np.random.uniform(low=-bound, high=bound, size=[L[i],L[i+1]])) \n",
    "                biases.append(np.random.uniform(low=-bound, high=bound, size=[1, L[i+1]]))  \n",
    "            else:\n",
    "                std = math.sqrt(2/(nin+nout)) \n",
    "                weights.append(np.random.normal(loc=0.0, scale=std, size=[L[i],L[i+1]])) \n",
    "                biases.append(np.random.normal(loc=0.0, scale=std, size=[1, L[i+1]]))       \n",
    "        \n",
    "    return (weights, biases)\n",
    "# Initializes the unit test neural network\n",
    "L_UT  = [3, 5, 1]\n",
    "ACT_UT = [ReLU, Tanh]\n",
    "NN_Glorot = init_NN_Glorot(L_UT, ACT_UT)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 59,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "([array([[ 0.7156774 ,  0.52084032, -0.84287379,  0.14508935, -0.54654787],\n",
       "         [-0.26486784,  0.08809447,  0.43148412, -0.46002952, -1.20048007],\n",
       "         [ 0.09322573,  1.3651345 , -0.10474934,  0.02422703,  0.92030534]]),\n",
       "  array([[-0.42420221],\n",
       "         [ 0.59666744],\n",
       "         [ 0.05814902],\n",
       "         [ 0.03165296],\n",
       "         [-0.02853249]])],\n",
       " [array([[ 0.3048165 ,  0.11198656,  0.22830806,  0.12922541, -0.58856666]]),\n",
       "  array([[0.24699705]])])"
      ]
     },
     "execution_count": 59,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "NN_Glorot"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Numpy einsum (EINstein SUMmation)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "[Einsum](https://docs.scipy.org/doc/numpy/reference/generated/numpy.einsum.html) gives us the possibility to compute almost any matrix operation in a single function. You can find a good description in the link above. Here are a few examples of some important uses:\n",
    "\n",
    "**Transpose:** We can write the transpose of matrix $A$:\n",
    "\n",
    "```\n",
    "np.einsum('ij -> ji', A) \n",
    "```\n",
    "\n",
    "**Trace:** We can write the trace of matrix $A$:\n",
    "\n",
    "```\n",
    "np.einsum('ii -> ', A) \n",
    "```\n",
    "\n",
    "**Diagonal:** We can write the diagonal of matrix $A$:\n",
    "\n",
    "```\n",
    "np.einsum('ii -> i', A) \n",
    "```\n",
    " \n",
    "**Matrix product:** We can write the multiplication of matrices $A$ and $B$ as:\n",
    "\n",
    "```\n",
    "np.einsum('ij, jk -> ik', A, B)\n",
    "```\n",
    "\n",
    "Note that $j$ in both matrices $A$ and $B$ should be the same size. \n",
    "\n",
    "**Batched matrix product (or why bothering):** All of the functions we performed above are built in numpy (np.tranpose, np.trace, np.matmul), however, when you want to do more complex operations, it might become less readable and computationaly efficient. Let's introduce a three dimensional matrix $H$ with indices $b,j,k$, where the first dimension is the batch (training example) dimension. In einsum, we can then write:\n",
    "\n",
    "```\n",
    "np.einsum('ij, bjk -> bik', A, H)\n",
    "```\n",
    "\n",
    "In order to perform a batched matrix multiplication where we multiple over the second dimension in the first marix and second dimension in the second matrix. The result is a new three dimensional matrix where the first dimension is the first dimension from $H$ and second is the first dimension from $A$ and last dimension the last dimension from $H$. This is a very simple one line (and readable) way to do matrix operations that will be very useful for neural network code. \n",
    "\n",
    "\n",
    "#### _**Tips and tricks when using einsum**_\n",
    "\n",
    "At the beginning, einsum might be a bit difficult to work with. The most important thing to do when using it is keeping track of the dimensions of your input and output matrices. An easy way to keep track of these dimensions is by using some sort of naming convention. Just like in the batched matrix product above we used $b$ to denote the batch dimension. In all the functions of this notebook, we leave some convention of names of indexes for the einsum in the explanation of the functions. We hope you find them useful!\n",
    "\n",
    "There are some other useful resources to understand numpy.einsum:\n",
    "\n",
    "* [Olexa Bilaniuk's great blogpost on einsum]( https://obilaniu6266h16.wordpress.com/2016/02/04/einstein-summation-in-numpy/ )\n",
    "* [Stackoverflow answer to: Understanding NumPy's einsum]( https://stackoverflow.com/q/26089893/8899404 )\n",
    "* [Jessica Stringham post on einsum]( https://jessicastringham.net/2018/01/01/einsum/ )\n",
    "* [Slides of einstein summation from oxford]( http://www-astro.physics.ox.ac.uk/~sr/lectures/vectors/lecture10final.pdfc )"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Forward pass\n",
    "\n",
    "The forward pass has been implemented for you. Please note how we have used einsum to perform the affine tranformation."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "collapsed": true
   },
   "outputs": [],
   "source": [
    "def forward_pass(x, NN, activations):\n",
    "    \"\"\"\n",
    "    This function performs a forward pass recursively. It saves lists for both affine transforms of units (z) and activated units (a)\n",
    "    Input:\n",
    "    x: The input of the network             (np.array of shape: (batch_size, number_of_features))\n",
    "    NN: The initialized neural network      (tuple of list of matrices)\n",
    "    activations: the activations to be used (list of functions, same len as NN)\n",
    "\n",
    "    Output:\n",
    "    a: A list of affine transformations, that is, all x*w+b.\n",
    "    z: A list of activated units (ALL activated units including input and output).\n",
    "    \n",
    "    Shapes for the einsum:\n",
    "    b: batch size\n",
    "    i: size of the input hidden layer (layer l)\n",
    "    o: size of the output (layer l+1)\n",
    "    \"\"\"\n",
    "    z = [x]\n",
    "    a = []\n",
    "        \n",
    "    for l in range(len(NN[0])):\n",
    "        a.append(np.einsum('bi, io -> bo', z[l], NN[0][l]) + NN[1][l])  # The affine transform x*w+b\n",
    "        z.append(activations[l](a[l]))                                  # The non-linearity    \n",
    "    \n",
    "    return a, z"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Forward pass unit test\n",
    "\n",
    "Below is a piece of code that takes a very particular setting of the network and inputs and test whether it gives the expected results."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 82,
   "metadata": {},
   "outputs": [],
   "source": [
    "ACT_F_UT = [Linear, Linear]\n",
    "test_a, test_z = forward_pass(np.array([[1,1,1]]), NN_UT, ACT_F_UT) # input has shape (1, 3) 1 batch, 3 features\n",
    "\n",
    "# Checking shapes consistency\n",
    "assert np.all(test_z[0]==np.array([1,1,1])) # Are the input vector and the first units the same?\n",
    "assert np.all(test_z[1]==test_a[0])         # Are the first affine transformations and hidden units the same?\n",
    "assert np.all(test_z[2]==test_a[1])         # Are the output units and the affine transformations the same?\n",
    "\n",
    "# Checking correctnes of values\n",
    "# First layer, calculate np.sum(np.array([1,1,1])*np.array([1,1,1]))+1 = 4\n",
    "assert np.all(test_z[1] == 4.)\n",
    "# Second layer, calculate np.sum(np.array([4,4,4,4,4])*np.array([1,1,1,1,1]))+1 = 21\n",
    "assert np.all(test_z[2] == 21.)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Loss functions"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "In order to perform a backward pass we need to define a loss function and its derivative with respect to the output of the neural network $y$"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "collapsed": true
   },
   "outputs": [],
   "source": [
    "def squared_error(t, y, derivative=False):\n",
    "    \"\"\"\n",
    "    Computes the squared error function and its derivative \n",
    "    Input:\n",
    "    t:      target (expected output)          (np.array)\n",
    "    y:      output from forward pass (np.array, must be the same shape as t)\n",
    "    derivative: whether to return the derivative with respect to y or return the loss (boolean)\n",
    "    \"\"\"\n",
    "    if np.shape(t)!=np.shape(y):\n",
    "        print(\"t and y have different shapes\")\n",
    "    if derivative: # Return the derivative of the function\n",
    "        return (y-t)\n",
    "    else:\n",
    "        return 0.5*(y-t)**2"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Exercise d) Implement cross entropy loss\n",
    "\n",
    "Insert code below to implement cross-entropy loss for general dimensionality of $t$."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "collapsed": true
   },
   "outputs": [],
   "source": [
    "def cross_entropy_loss(t, y, derivative=False):\n",
    "    \"\"\"\n",
    "    Computes the cross entropy loss function and its derivative \n",
    "    Input:\n",
    "    t:      target (expected output)          (np.array)\n",
    "    y:      output from forward pass (np.array, must be the same shape as t)\n",
    "    derivative: whether to return the derivative with respect to y or return the loss (boolean)\n",
    "    \"\"\"\n",
    "    ## Insert code here"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Backward pass "
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Exercise e) Complete code for backward pass\n",
    "\n",
    "Below is a implementation of the backward pass with some lines removed. Insert the missing lines."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "collapsed": true
   },
   "outputs": [],
   "source": [
    "def backward_pass(x, t, y, z, a, NN, activations, loss):\n",
    "    \"\"\"\n",
    "    This function performs a backward pass ITERATIVELY. It saves lists all of the derivatives in the process\n",
    "    \n",
    "    Input:\n",
    "    x:           The input used for the batch                (np.array)\n",
    "    t:           The observed targets                        (np.array, the first dimension must be the same to x)\n",
    "    y:           The output of the forward_pass of NN for x  (np.array, must have the same shape as t)\n",
    "    a:           The affine transforms from the forward_pass (np.array)\n",
    "    z:           The activated units from the forward_pass (np.array)\n",
    "    activations: The activations to be used                  (list of functions)\n",
    "    loss:        The loss function to be used                (one function)\n",
    "    \n",
    "    Output:\n",
    "    g_w: A list of gradients for every hidden unit \n",
    "    g_b: A list of gradients for every bias\n",
    "    \n",
    "    Shapes for the einsum:\n",
    "    b: batch size\n",
    "    i: size of the input hidden layer (layer l)\n",
    "    o: size of the output (layer l+1)\n",
    "    \"\"\"\n",
    "    BS = x.shape[0] # Implied batch shape \n",
    "    \n",
    "    # First, let's compute the list of derivatives of z with respect to a \n",
    "    d_a = []\n",
    "    for i in range(len(activations)):\n",
    "        d_a.append(activations[i](a[i], derivative=True))\n",
    "    \n",
    "    # Second, let's compute the derivative of the loss function\n",
    "    t = t.reshape(BS, -1)\n",
    "    \n",
    "    d_loss = 0 # <- Insert correct expression here++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n",
    "    \n",
    "     \n",
    "    # Third, let's compute the derivative of the biases and the weights\n",
    "    g_w   = [] # List to save the gradient of the weights\n",
    "    g_b   = [] # List to save the gradients of the biases\n",
    "\n",
    "    delta = np.einsum('bo, bo -> bo', d_loss, d_a[-1])# loss shape: (b, o); pre-activation units shape: (b, o) hadamard product\n",
    "\n",
    "    g_b.append(np.mean(delta, axis=0))\n",
    "    g_w.append(np.mean(np.einsum('bo, bi -> bio', delta, z[-2]), axis=0)) # delta shape: (b, o), activations shape: (b, h) \n",
    "\n",
    "    for l in range(1, len(NN[0])):\n",
    "        d_C_d_z = np.einsum('bo, io -> bi', delta, NN[0][-l])  # Derivative of the Cost with respect to an activated layer d_C_d_z. \n",
    "                                                               #  delta shape: as above; weights shape: (i, o)\n",
    "                                                               # Delta: d_C_d_z (element-wise mult) derivative of the activation layers\n",
    "                                                               #  delta shape: as above; d_z shape: (b, i)  \n",
    "        delta = 0   # <- Insert correct expression ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n",
    "                                                                \n",
    "        g_b.append(np.mean(delta, axis=0)) \n",
    "        g_w.append(np.mean(np.einsum('bo, bi -> bio', delta, z[-l-2]), axis=0)) # Derivative of cost with respect to weights in layer l:\n",
    "                                                                                # delta shape: as above; activations of l-1 shape: (b, i)\n",
    "    \n",
    "    return g_b[::-1], g_w[::-1]"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Backward pass unit test"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "We are going to perform the unit test of the backward pass with a finite difference estimation, make sure to read the description of the function and that you understand it well:"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Exercise f) Test correctness of derivatives with finite difference method\n",
    "\n",
    "Write a small function that uses [the finite difference method](https://en.wikipedia.org/wiki/Finite_difference_method) to test whether the backpropation implementation is working. In short we will use\n",
    "$$\n",
    "\\frac{\\partial E(w)}{\\partial w_{ij}^{(l)}} \\approx \\frac{E(v)-E(w)}{dw}\n",
    "$$\n",
    "for $dw \\ll 1$ and $v$ is the same network as $w$ apart from $v_{ij}^{(l)} = w_{ij}^{(l)} + dw$.\n",
    "\n",
    "As arguments the function should take: some data $x$ and $t$ as in the example above, the network including activations, the indices $i$, $j$, $l$ of the weight we investigate and $dw$ and return the right hand side of the expression above.\n",
    "\n",
    "_Insert your code in the cell below._\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "collapsed": true
   },
   "outputs": [],
   "source": [
    "# Insert your finite difference code here"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Once you have implemented the function you can compare this number with the left hand side computed by the implementation above.\n",
    "\n",
    "Try for different parameters and different values of $dw$. Scan over a range of $dw$ values. Why does the method break down for really small $dw$?\n",
    "\n",
    "_Insert your written answer here._\n",
    "\n",
    "Finite differences gives us gradients without computing gradients explicitly. Why don't we use it in practice then?\n",
    "\n",
    "_Insert your written answer here._"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Below is reference code that computes the finite differences for all parameters."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def finite_diff_grad(x, NN, ACT_F, epsilon=None):\n",
    "    \"\"\"\n",
    "    Finite differences gradient estimator: https://en.wikipedia.org/wiki/Finite_difference_method\n",
    "    The idea is that we can approximate the derivative of any function (f) with respect to any argument (w) by evaluating the function at (w+e)\n",
    "    where (e) is a small number and then computing the following opertion (f(w+e)-f(w))/e . Note that we would need N+1 evaluations of\n",
    "    the function in order to compute the whole Jacobian (first derivatives matrix) where N is the number of arguments. The \"+1\" comes from the\n",
    "    fact that we also need to evaluate the function at the current values of the argument.\n",
    "    \n",
    "    Input:\n",
    "    x:       The point at which we want to evaluate the gradient\n",
    "    NN:      The tuple that contains the neural network\n",
    "    ACT_F:   The activation functions in order to perform the forward pass\n",
    "    epsilon: The size of the difference\n",
    "    \n",
    "    Output:\n",
    "    Two lists, the first one contains the gradients with respect to the weights, the second with respect to the biases\n",
    "    \"\"\"\n",
    "    from copy import deepcopy\n",
    "    \n",
    "    if epsilon == None:\n",
    "        epsilon = np.finfo(np.float32).eps # Machine epsilon for float 32\n",
    "        \n",
    "    grads = deepcopy(NN)               # Copy of structure of the weights and biases to save the gradients                        \n",
    "    _ , test_z = forward_pass(x, NN_UT, ACT_F_UT) # We evaluate f(x)\n",
    "    \n",
    "    for e in range(len(NN)):                       # Iterator over elements of the NN:       weights or biases\n",
    "        for h in range(len(NN[e])):                # Iterator over the layer of the element: layer number\n",
    "            for r in range(NN[e][h].shape[0]):     # Iterator over                           row number\n",
    "                for c in range(NN[e][h].shape[1]): # Iterator over                           column number \n",
    "                    NN_copy             = deepcopy(NN)    \n",
    "                    NN_copy[e][h][r,c] += epsilon\n",
    "                    _, test_z_eps       = forward_pass(x, NN_copy, ACT_F)     # We evaluate f(x+eps)\n",
    "                    grads[e][h][r,c]    = (test_z_eps[-1]-test_z[-1])/epsilon # Definition of finite differences gradient\n",
    "    \n",
    "    return grads[0], grads[1]"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "collapsed": true,
    "scrolled": true
   },
   "outputs": [],
   "source": [
    "### Unit test \n",
    "\n",
    "## First lest's compute the backward pass using our own function\n",
    "# Forward pass\n",
    "test_a, test_z = forward_pass(np.array([[1,1,1]]), NN_UT, ACT_F_UT)\n",
    "# Backward pass\n",
    "test_g_b, test_g_w = backward_pass(np.array([[1,1,1]]), np.array([20]), test_a[-1], test_z, test_a, NN_UT, ACT_F_UT, squared_error)\n",
    "# Estimation by finite differences\n",
    "test_fdg_w, test_fdg_b = finite_diff_grad(np.array([[1,1,1]]), NN_UT, ACT_F_UT)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "collapsed": true
   },
   "outputs": [],
   "source": [
    "# Test whether the weights and biases are all equal as the ones we estimated using back propagation\n",
    "for l in range(len(test_g_w)):\n",
    "    assert np.allclose(test_fdg_w[l], test_g_w[l])\n",
    "    assert np.allclose(test_fdg_b[l], test_g_b[l])"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Training and validation\n",
    "\n",
    "We are ready to train some neural networks! Below we give some example initializations and a training loop. Try it out. "
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "collapsed": true
   },
   "outputs": [],
   "source": [
    "# Initialize an arbitrary neural network\n",
    "#L  = [3, 16, 1]\n",
    "L  = [1, 8, 1]\n",
    "NN = init_NN(L)\n",
    "#NN = init_NN_glorot(L, uniform=True)\n",
    "#NN = init_NN_he_ReLU(L, uniform=True)\n",
    "\n",
    "ACT_F = [ReLU, Linear]\n",
    "#ACT_F = [Tanh, Linear]\n",
    "\n",
    "# Recommended hyper-parameters for 1-D: \n",
    "# L  = [1, 8, 1]\n",
    "# EPOCHS = 10000\n",
    "# BATCH_SIZE = 128 \n",
    "# LEARN_R = 2.5e-1 for Tanh and LEARN_R = 1e-1 for ReLU\n",
    "\n",
    "# Recommended hyper-parameters for 3-D: \n",
    "# L  = [3, 16, 1] \n",
    "# EPOCHS = 10000\n",
    "# BATCH_SIZE = 128 \n",
    "# LEARN_R = 5e-2 for ReLU and LEARN_R = 1e-1 for Tanh\n",
    "\n",
    "### Notice that, when we switch from tanh to relu activation, we decrease the learning rate. This is due the stability of the gradients \n",
    "## of the activation functions."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "collapsed": true
   },
   "outputs": [],
   "source": [
    "# Initialize training hyperparameters\n",
    "EPOCHS = 20000\n",
    "BATCH_SIZE = 128 \n",
    "LEARN_R = 1e-2 "
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "scrolled": true
   },
   "outputs": [],
   "source": [
    "train_loss = []\n",
    "val_loss = []\n",
    "\n",
    "for e in range(EPOCHS):\n",
    "    # Mini-batch indexes\n",
    "    idx = np.random.choice(x_train.shape[0], size=BATCH_SIZE)\n",
    "    # Forward pass\n",
    "    aff, units = forward_pass(x_train[idx,:], NN, ACT_F)\n",
    "    # Backward pass\n",
    "    g_b, g_w = backward_pass(x_train[idx,:], y_train[idx], units[-1], units, aff, NN, ACT_F, squared_error)\n",
    "    \n",
    "    # Stochastic gradient descent\n",
    "    for l in range(len(g_b)):\n",
    "        NN[0][l] -= LEARN_R*g_w[l]\n",
    "        NN[1][l] -= LEARN_R*g_b[l]\n",
    "        \n",
    "    # Training loss\n",
    "    _, units = forward_pass(x_train, NN, ACT_F)\n",
    "    # Estimate loss function\n",
    "    #print(np.max(squared_error(y_train, units[-1])))\n",
    "    train_loss.append(np.mean(squared_error(y_train, np.squeeze(units[-1]))))\n",
    "    \n",
    "    # Validation\n",
    "    # Forward pass\n",
    "    _, units = forward_pass(x_validation, NN, ACT_F)\n",
    "    # Estimate validation loss function\n",
    "    val_loss.append(np.mean(squared_error(y_validation, np.squeeze(units[-1]))))\n",
    "    \n",
    "    if e%500==0:\n",
    "        print(\"{:4d}\".format(e),\n",
    "              \"({:5.2f}%)\".format(e/EPOCHS*100), \n",
    "              \"Train loss: {:4.3f} \\t Validation loss: {:4.3f}\".format(train_loss[-1], val_loss[-1]))\n",
    "        \n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "plt.plot(range(len(train_loss)), train_loss);\n",
    "plt.plot(range(len(val_loss)), val_loss);"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Testing\n",
    "\n",
    "We have kept the calculation of the test error separate in order to emphasize that you should not use the test set in optimization."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "_, units = forward_pass(x_test, NN, ACT_F)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "plt.scatter(y_test, units[-1]);\n",
    "plt.plot([np.min(y_test), np.max(y_test)], [np.min(y_test), np.max(y_test)], color='k');\n",
    "plt.xlabel(\"y\");\n",
    "plt.ylabel(\"$\\hat{y}$\");\n",
    "plt.title(\"Model prediction vs real in the test set, the close to the line the better\")\n",
    "plt.grid(True);\n",
    "plt.axis('equal');\n",
    "plt.tight_layout();\n",
    "\n",
    "print(\"Test loss:  {:4.3f}\".format(np.mean(squared_error(y_test, np.squeeze(units[-1])))))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if D1:\n",
    "    plt.scatter(x_train[:,0], y_train, label=\"train data\");\n",
    "    plt.scatter(x_test[:,0], units[-1], label=\"test prediction\");\n",
    "    plt.scatter(x_test[:,0], y_test, label=\"test data\");\n",
    "    plt.legend();\n",
    "    plt.xlabel(\"x\");\n",
    "    plt.ylabel(\"y\");\n",
    "else:\n",
    "    plt.scatter(x_train[:,1], y_train, label=\"train data\");\n",
    "    plt.scatter(x_test[:,1], units[-1], label=\"test data prediction\");\n",
    "    plt.scatter(x_test[:,1], y_test, label=\"test data\");\n",
    "    plt.legend();\n",
    "    plt.xlabel(\"x\");\n",
    "    plt.ylabel(\"y\");"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Exercise g) Show overfitting, underfitting and just right fitting\n",
    "\n",
    "Vary the architecture and other things to show clear signs of overfitting (=training loss significantly lower than test loss) and underfitting (=not fitting enoung to training data so that test performance is also hurt).\n",
    "\n",
    "See also if you can get a good compromise which leads to a low validation loss. \n",
    "\n",
    "For this problem do you see any big difference between validation and test loss? The answer here will probably be no. Discuss cases where it is important to keep the two separate.\n",
    "\n",
    "_Insert written answer here._\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "collapsed": true
   },
   "outputs": [],
   "source": [
    "# Insert your code for getting overfitting, underfitting and just right fitting"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Next steps - classification\n",
    "\n",
    "It is straight forward to extend what we have done to classification. \n",
    "\n",
    "For numerical stability it is better to make softmax and cross-entropy as one function so we write the cross entropy loss as a function of the logits we talked about last week. \n",
    "\n",
    "Next week we will see how to perform classification in PyTorch."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Exercise h) optional - Implement backpropagation for classification\n",
    "\n",
    "Should be possible with very few lines of code. :-)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "collapsed": true
   },
   "outputs": [],
   "source": [
    "# Just add code."
   ]
  }
 ],
 "metadata": {
  "anaconda-cloud": {},
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.7.4"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}

NameError: name 'null' is not defined